## Import des librairies

In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import csv
from functions import split
import matplotlib.pyplot as plt

## Préparation

In [ ]:
# CREATION DE L'INSTANCE "SPLIT"
splitter_object = split.splitter("./data", '\t')

splitter_object.file_splitter("title.principals.tsv", "category", "title.principals")
splitter_object.file_splitter("title.akas.tsv", "region", "title.akas")
splitter_object.file_splitter("title.basics.tsv", "titleType", "title.basics")

# La liste des acteurs par film

In [ ]:
# CREATION ET MODIFICATION DES DATAFRAMES "actor" et "actress"
global_actor = pd.read_table("./data/CURATED/title.principals/actor.csv", sep=',')
global_actress = pd.read_table("./data/CURATED/title.principals/actress.csv", sep=',')
df_actor_actress = pd.concat([global_actor,global_actress])
df_actor_actress.drop(["ordering","category","job","characters"], axis=1,inplace=True)

In [ ]:
# CREATION ET MODIFICATION DES DATAFRAMES "movie"
global_movie = pd.read_table("./data/CURATED/title.basics/movie.csv", sep=',')
df_movie = global_movie.drop(["titleType","primaryTitle","isAdult","startYear","endYear","runtimeMinutes","genres"], axis=1)

# CREATION ET MODIFICATION DES DATAFRAMES "name_actor"
global_name_actor = pd.read_table("./data/RAW/name.basics.tsv", sep='\t')
df_name_actor = global_name_actor.dropna()
df_name_actor = df_name_actor[["nconst","primaryName"]][df_name_actor["primaryProfession"].str.contains("actor|actress", regex=True)]

# CHANGEMENT DES INDEX
df_actor_actress.set_index("tconst", inplace=True)
df_movie.set_index("tconst", inplace=True)
df_name_actor.set_index("nconst", inplace=True)

In [ ]:
# FUSION ENTRE "df_actor_actress" ET "df_movie"
df_join_actor_movie = df_actor_actress.join(df_movie)
df_join_actor_movie.set_index("nconst", inplace=True)

# FUSION DES DATAFRAMES "df_join_actor_movie" ET "df_name_actor"
df_join_total = df_join_actor_movie.join(df_name_actor)
df_join_total = df_join_total.reset_index()
df_final = df_join_total.drop(["nconst"], axis=1)

In [ ]:
# CREATION DU CSV
df_final.to_csv("./data/CSV_FINI/df_final.csv")

In [ ]:
## SUPRESSION DES DATAFRAMES QUI NE SONT PLUS NECESSAIRES
del df_actor_actress
del df_join_actor_movie
del df_join_total
del df_movie
del df_name_actor

# La liste des films Américains (en gardant leur nom en français) et leur note moyenne

In [ ]:
# CREATION ET MODIFICATION DU DATAFRAME "rating"
global_rating = pd.read_csv("./data/RAW/title.ratings.tsv", sep='\t')
df_rating = global_rating.drop(["numVotes"], axis=1)
df_rating = df_rating.set_index("tconst")

In [ ]:
# CREATION ET MODIFICATION DU DATAFRAME "us"
global_us = pd.read_csv("data/CURATED/title.akas/US.csv")
df_us = global_us.dropna()
df_us = df_us.drop(["ordering","title","region","language","types","attributes","isOriginalTitle"],axis=1)
df_us.set_index("titleId", inplace=True)

In [ ]:
# FUSION DES DATAFRAMES "rating" ET "us"
df_rating_us = df_us.join(df_rating)

In [ ]:
# CREATION ET MODIFICATION DU DATAFRAME "basics"
global_basics = pd.read_csv("data/CURATED/title.basics/movie.csv")
df_basics = global_basics.dropna()
df_basics = df_basics.drop(["titleType","primaryTitle","isAdult","startYear","endYear","runtimeMinutes","genres"],axis=1)
df_basics.set_index("tconst", inplace=True)

In [ ]:
# FUSION DES DATAFRAMES "rating_us" ET "basics"
df_total2 = df_rating_us.join(df_basics)
df_final2 = df_total2.dropna()

In [ ]:
# CREATION DU CSV
df_final2.to_csv("./data/CSV_FINI/df_final2.csv")

In [ ]:
# SUPRESSION DES DATAFRAMES QUI NE SONT PLUS NECESSAIRES
del df_basics
del df_rating_us
del df_rating
del df_total2
del df_us

# Les notes moyennes des différents genres

In [ ]:
# CREATION D'UNE COPIE DE "global_basics"
df_basics = global_basics

In [ ]:
# SPLIT DE LA COLONNE "genres"
df_basics_genre = df_basics["genres"].str.split(",", expand = True)

In [ ]:
# FUSION DES DATAFRAMES "df_basics" ET "df_basics_genre"
df_genre_basics_split = df_basics.join(df_basics_genre)

In [ ]:
# MODIFICATION DU DATAFRAME "df_genre_basics_split"
df_genre_basics_split = df_genre_basics_split.drop(["genres"], axis=1)
df_genre_basics_split.rename(columns={0: "genre1", 1: "genre2", 2: "genre3"}, inplace=True)

In [ ]:
# CREATION ET MODIFICATION DU DATAFRAME "df_rating"
df_rating = global_rating
df_genre_basics_split.set_index("tconst", inplace=True)
df_rating.set_index("tconst", inplace=True)
df_genre_rating = df_genre_basics_split.join(df_rating)
df_genre_rating.drop(["titleType","primaryTitle","originalTitle", "isAdult", "startYear","endYear","runtimeMinutes","numVotes"], axis=1, inplace=True)

In [ ]:
# CREATION DES DATAFRAMES CONTENANTS LES GROUPBY DES DIFFERENTES COLONNES "genre1","genre2" et "genre3"
df_genre1 = df_genre_rating.groupby("genre1").mean()
df_genre2 = df_genre_rating.groupby("genre2").mean()
df_genre3 = df_genre_rating.groupby("genre3").mean()

In [ ]:
# FUSION DES DATAFRAMES "df_genre1","df_genre2" ET "df_genre3"
df_genre_total = pd.concat([df_genre1,df_genre2,df_genre3], axis=1, keys=["note1","note2","note3"])

In [ ]:
# MOYENNE DES 3 DATAFRAMES CI-DESSUS
df_final3 = df_genre_total.mean(axis = 1)

In [ ]:
# CREATION DU CSV
df_final3.to_csv("./data/CSV_FINI/df_final3.csv")

In [ ]:
# SUPRESSION DES DATAFRAMES QUI NE SONT PLUS NECESSAIRES
del df_rating
del df_genre_total
del df_genre_rating
del df_genre_basics_split
del df_genre1
del df_genre2
del df_genre3
del df_basics_genre
del df_basics
del a
